In [1]:
import pandas as pd

df = pd.read_csv('../../00_Uncleaned_Data/Target/100050.csv', delimiter=';')    
df.head()
len(df)

25291

In [26]:
df = df.drop(columns=['geo_point_2d', 'timestamp_text'])
df.head()

,Datum/Zeit,pm10_stundenmittelwerte_ug_m3,pm2_5_stundenmittelwerte_ug_m3,no2_stundenmittelwerte_ug_m3
0,2023-08-15T11:00:00+00:00,13.915,8.667,28.570
1,2023-08-15T14:00:00+00:00,15.811,9.252,38.085
2,2023-08-15T17:00:00+00:00,14.185,9.546,28.069
3,2023-08-15T18:00:00+00:00,15.112,10.502,36.935
4,2023-08-15T20:00:00+00:00,14.407,10.177,24.552


In [27]:
# Convert 'Datum/Zeit' to datetime
df['Datum/Zeit'] = pd.to_datetime(df['Datum/Zeit'])

# Set 'Datum/Zeit' as the index
df.set_index('Datum/Zeit', inplace=True)

# Reindex to include all possible timestamps within the range
df = df.reindex(full_range)

# Extract weekday and hour
df['weekday'] = df.index.weekday
df['hour'] = df.index.hour

# Function to fill missing values with mean of the same weekday and hour
def fill_missing_with_mean(df, column):
    return df[column].fillna(df.groupby(['weekday', 'hour'])[column].transform('mean'))

# Fill missing values
df['pm10_stundenmittelwerte_ug_m3'] = fill_missing_with_mean(df, 'pm10_stundenmittelwerte_ug_m3')
df['pm2_5_stundenmittelwerte_ug_m3'] = fill_missing_with_mean(df, 'pm2_5_stundenmittelwerte_ug_m3')
df['no2_stundenmittelwerte_ug_m3'] = fill_missing_with_mean(df, 'no2_stundenmittelwerte_ug_m3')

# Drop the temporary columns
df = df.drop(columns=['weekday', 'hour'])

# Reset the index
df.reset_index(inplace=True)

df

,index,pm10_stundenmittelwerte_ug_m3,pm2_5_stundenmittelwerte_ug_m3,no2_stundenmittelwerte_ug_m3
0,2021-12-31 23:00:00+00:00,52.595,59.614,23.796
1,2022-01-01 00:00:00+00:00,85.530,58.362,22.670
2,2022-01-01 01:00:00+00:00,72.819,50.665,21.628
3,2022-01-01 02:00:00+00:00,64.542,45.385,20.943
4,2022-01-01 03:00:00+00:00,48.021,35.039,20.919
...,...,...,...,...
25360,2024-11-22 15:00:00+00:00,5.242,3.733,13.962
25361,2024-11-22 16:00:00+00:00,4.178,3.183,11.757
25362,2024-11-22 17:00:00+00:00,5.640,4.421,13.571
25363,2024-11-22 18:00:00+00:00,5.590,4.363,9.202


In [ ]:
df.to_csv('../02_Cleaned/Target/1000050cleaned.csv', index=False)